<a href="https://colab.research.google.com/github/adlihs/PyTorch/blob/main/Solving_Problems_with_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Person**: A bike delivery person with a 7-mile delivery order.

**Situation**: The company promises delivery in under 30 minutes and one more late delivery could put the job at risk.

**Question**: Can the bike delivery person makes this delivery on time?


**Next Steps based on ML pipeline:**


*   **Prepare** delivery data, the distances and times from past orders
*   **Build** a simple neural network using Pytorch
*   **Train** it to find the relationship between distance and delivery time.
*   **Predict** whether the delivery person can make that 7-mile delivery in time




## Imports

In [ ]:
!pip install helper_utils

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

import helper_utils

torch.manual_seed(42)